In [2]:
import json, os, csv
from collections import defaultdict

with open('advancements/everything.json', 'r') as f:
    adv_data = json.load(f)

In [5]:
filtered = defaultdict(lambda:[])
for x in list(adv_data.keys()):
    if x == "DataVersion":
        continue
    if x[:17] != 'minecraft:recipes':
        tab, adv = x.split('/')
        filtered[tab].append(adv)

In [6]:
for tab in filtered:
    print(tab, len(filtered[tab]))

advancements = [
    ['display_name', 'true_name']
]

for root in ['minecraft', 'blazeandcave']:
    folder_path = f'data/{root}/advancements'
    for tab in os.listdir(folder_path):
        tab_path = os.path.join(folder_path, tab)
        root = folder_path.split('/')[1]
        prefix = f'{root}:{tab}'
        for adv in os.listdir(tab_path):
            true_name = f'{prefix}/{adv[:-5]}'
            adv_path = os.path.join(tab_path, adv)
            try:
                with open(adv_path, 'r') as f:
                    data = json.load(f)
                    display_name = data['display']['title']['translate']
            except KeyError:
                display_name = "NOT AN ADVANCEMENT"
            advancements.append([display_name, true_name])

with open('advname_to_path.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    writer.writerows(advancements)

print("Printing Missing Achievements")
# print(adv_data.keys())
for row in advancements[1:]:
    if row[1] not in adv_data:
        print(row[1])

blazeandcave:biomes 59
blazeandcave:challenges 38
blazeandcave:bacap 23
minecraft:adventure 35
minecraft:story 16
blazeandcave:statistics 60
blazeandcave:technical 44
blazeandcave:mining 92
blazeandcave:weaponry 44
blazeandcave:monsters 75
blazeandcave:enchanting 49
blazeandcave:redstone 45
blazeandcave:adventure 100
blazeandcave:animal 113
minecraft:nether 24
blazeandcave:nether 82
blazeandcave:farming 57
minecraft:husbandry 27
blazeandcave:end 35
blazeandcave:building 105
blazeandcave:potion 24
minecraft:end 9
Printing Missing Achievements
blazeandcave:technical/a_wizards_breakfast
blazeandcave:technical/a_wizards_breakfast_fail
blazeandcave:technical/below_void
blazeandcave:technical/castaway_fail
blazeandcave:technical/castaway_start
blazeandcave:technical/fail_dragon_vs
blazeandcave:technical/give_piglin_gold
blazeandcave:technical/just_keeps_going_fail
blazeandcave:technical/just_keeps_going_start
blazeandcave:technical/kill_dragon
blazeandcave:technical/kill_wither
blazeandcave:

In [7]:
# For generating a list of all the items, and their mapping to adv
stack_blocks = set(adv_data["blazeandcave:challenges/stack_all_the_blocks"]['criteria'].keys())
stack_items = set(adv_data["blazeandcave:challenges/stack_all_the_items"]['criteria'].keys())

blocks = set(adv_data["blazeandcave:challenges/all_the_blocks"]['criteria'].keys())
items = set(adv_data["blazeandcave:challenges/all_the_items"]['criteria'].keys())

print(len(stack_items), len(stack_blocks), len(stack_items.union(stack_blocks)))
print(len(items), len(blocks), len(items.union(blocks)))
print(items.union(blocks).difference(stack_items.union(stack_blocks)))
all_items = items.union(blocks)
stack_all_items = stack_items.union(stack_blocks)

everything = all_items.union(stack_all_items)

item_to_adv = []

for item in list(everything):
    adv = None
    if item in stack_blocks:
        adv = "blazeandcave:challenges/stack_all_the_blocks"
    elif item in stack_items:
        adv = "blazeandcave:challenges/stack_all_the_items"
    elif item in blocks:
        adv = "blazeandcave:challenges/all_the_blocks"
    elif item in items:
        adv = "blazeandcave:challenges/all_the_items"
    item_to_adv.append([item, adv])

item_to_adv = sorted(item_to_adv,  key=lambda x: x[0])

with open('item_to_adv.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    writer.writerows(item_to_adv)

285 818 1103
322 823 1145
{'spire_armor_trim', 'prize_pottery_sherd', 'heartbreak_pottery_sherd', 'raiser_armor_trim', 'burn_pottery_sherd', 'netherite_block', 'sentry_armor_trim', 'danger_pottery_sherd', 'plenty_pottery_sherd', 'dune_armor_trim', 'ward_armor_trim', 'snout_armor_trim', 'blade_pottery_sherd', 'skull_pottery_sherd', 'heart_pottery_sherd', 'host_armor_trim', 'silence_armor_trim', 'large_fern', 'recovery_compass', 'friend_pottery_sherd', 'shaper_armor_trim', 'sheaf_pottery_sherd', 'miner_pottery_sherd', 'vex_armor_trim', 'arms_up_pottery_sherd', 'eye_armor_trim', 'coast_armor_trim', 'snort_pottery_sherd', 'rib_armor_trim', 'netherite_upgrade', 'dragon_egg', 'wild_armor_trim', 'wayfinder_armor_trim', 'mourner_pottery_sherd', 'angler_pottery_sherd', 'brewer_pottery_sherd', 'howl_pottery_sherd', 'tide_armor_trim', 'tall_grass', 'deepslate_emerald_ore', 'explorer_pottery_sherd', 'shelter_pottery_sherd', 'archer_pottery_sherd'}


In [8]:
# Adding column for stack sizes
with open('data/blazeandcave/advancements/challenges/stack_all_the_blocks.json') as f:
    stack_blocks_json = json.load(f)['criteria']

with open('data/blazeandcave/advancements/challenges/stack_all_the_items.json') as f:
    stack_items_json = json.load(f)['criteria']

item_adv_stacksize = []
for item, adv in item_to_adv:
    stacksize = 1
    try:
        if item in stack_blocks_json:
            stacksize = stack_blocks_json[item]['conditions']['items'][0]['count']
        elif item in stack_items_json:
            stacksize = stack_items_json[item]['conditions']['items'][0]['count']
    except KeyError:
        pass
    name = ' '.join(word.capitalize() for word in item.split('_'))
    item_adv_stacksize.append([name, item, adv, stacksize])

item_adv_stacksize = sorted(item_adv_stacksize, key=lambda x: x[0])

with open('item_adv_stacksize.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter='\t')
    writer.writerows(item_adv_stacksize)

In [2]:
adv_col = []
with open('adv_col.csv') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        adv_col.append(row)

adv_to_cell = []
for i, adv in enumerate(adv_col):
    adv_to_cell.append([adv, f'E{i+'])

In [3]:
reference = '''IF(ISBLANK(A3), {"","",""}, {{A3, "Adv/Item", "Description"};IFERROR({FILTER({'All Items/Blocks'!$C$2:$D, 'All Items/Blocks'!$G$2:$G}, 'All Items/Blocks'!$B$2:$B=A3);{"","",""}}, {"", "", ""})})'''
full_string = reference

for i in range(20):
    row = i + 4
    full_string += ';' + reference.replace("A3", f'A{row}')

print(full_string)

IF(ISBLANK(A3), {"","",""}, {{A3, "Adv/Item", "Description"};IFERROR({FILTER({'All Items/Blocks'!$C$2:$D, 'All Items/Blocks'!$G$2:$G}, 'All Items/Blocks'!$B$2:$B=A3);{"","",""}}, {"", "", ""})});IF(ISBLANK(A4), {"","",""}, {{A4, "Adv/Item", "Description"};IFERROR({FILTER({'All Items/Blocks'!$C$2:$D, 'All Items/Blocks'!$G$2:$G}, 'All Items/Blocks'!$B$2:$B=A4);{"","",""}}, {"", "", ""})});IF(ISBLANK(A5), {"","",""}, {{A5, "Adv/Item", "Description"};IFERROR({FILTER({'All Items/Blocks'!$C$2:$D, 'All Items/Blocks'!$G$2:$G}, 'All Items/Blocks'!$B$2:$B=A5);{"","",""}}, {"", "", ""})});IF(ISBLANK(A6), {"","",""}, {{A6, "Adv/Item", "Description"};IFERROR({FILTER({'All Items/Blocks'!$C$2:$D, 'All Items/Blocks'!$G$2:$G}, 'All Items/Blocks'!$B$2:$B=A6);{"","",""}}, {"", "", ""})});IF(ISBLANK(A7), {"","",""}, {{A7, "Adv/Item", "Description"};IFERROR({FILTER({'All Items/Blocks'!$C$2:$D, 'All Items/Blocks'!$G$2:$G}, 'All Items/Blocks'!$B$2:$B=A7);{"","",""}}, {"", "", ""})});IF(ISBLANK(A8), {"","",""